![](https://www.pata.org/wp-content/uploads/2014/09/TripAdvisor_Logo-300x119.png)
# Predict TripAdvisor Rating
## В этом соревновании нам предстоит предсказать рейтинг ресторана в TripAdvisor
**По ходу задачи:**
* Прокачаем работу с pandas
* Научимся работать с Kaggle Notebooks
* Поймем как делать предобработку различных данных
* Научимся работать с пропущенными данными (Nan)
* Познакомимся с различными видами кодирования признаков
* Немного попробуем [Feature Engineering](https://ru.wikipedia.org/wiki/Конструирование_признаков) (генерировать новые признаки)
* И совсем немного затронем ML
* И многое другое...   



### И самое важное, все это вы сможете сделать самостоятельно!

*Этот Ноутбук являетсся Примером/Шаблоном к этому соревнованию (Baseline) и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.

> что такое baseline решение, зачем оно нужно и почему предоставлять baseline к соревнованию стало важным стандартом на kaggle и других площадках.   
**baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой, просто для примера. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline являеться хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

В контексте нашего соревнования baseline идет с небольшими примерами того, что можно делать с данными, и с инструкцией, что делать дальше, чтобы улучшить результат.  Вообще готовым решением это сложно назвать, так как используются всего 2 самых простых признака (а остальные исключаются).

# import

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
    
import re
import datetime

# Any results you write to the current directory are saved as output.

/kaggle/input/sf-dst-restaurant-rating/main_task.csv
/kaggle/input/sf-dst-restaurant-rating/kaggle_task.csv
/kaggle/input/sf-dst-restaurant-rating/sample_submission.csv


In [2]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [3]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# DATA

In [4]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
Restaurant_id        40000 non-null object
City                 40000 non-null object
Cuisine Style        30717 non-null object
Ranking              40000 non-null float64
Rating               40000 non-null float64
Price Range          26114 non-null object
Number of Reviews    37457 non-null float64
Reviews              40000 non-null object
URL_TA               40000 non-null object
ID_TA                40000 non-null object
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


In [6]:
df_train.head(5)

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963


In [7]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
Restaurant_id        10000 non-null object
City                 10000 non-null object
Cuisine Style        7693 non-null object
Ranking              10000 non-null float64
Price Range          6525 non-null object
Number of Reviews    9343 non-null float64
Reviews              9998 non-null object
URL_TA               10000 non-null object
ID_TA                10000 non-null object
dtypes: float64(2), object(7)
memory usage: 703.2+ KB


In [8]:
df_test.head(5)

,Restaurant_id,City,Cuisine Style,Ranking,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,id_0,Paris,"['Bar', 'Pub']",12963.0,$$ - $$$,4.0,"[[], []]",/Restaurant_Review-g187147-d10746918-Reviews-L...,d10746918
1,id_1,Helsinki,"['European', 'Scandinavian', 'Gluten Free Opti...",106.0,$$ - $$$,97.0,"[['Very good reviews!', 'Fine dining in Hakani...",/Restaurant_Review-g189934-d6674944-Reviews-Ra...,d6674944
2,id_2,Edinburgh,['Vegetarian Friendly'],810.0,$$ - $$$,28.0,"[['Better than the Links', 'Ivy Black'], ['12/...",/Restaurant_Review-g186525-d13129638-Reviews-B...,d13129638
3,id_3,London,"['Italian', 'Mediterranean', 'European', 'Vege...",1669.0,$$$$,202.0,"[['Most exquisite', 'Delicious and authentic']...",/Restaurant_Review-g186338-d680417-Reviews-Qui...,d680417
4,id_4,Bratislava,"['Italian', 'Mediterranean', 'European', 'Seaf...",37.0,$$$$,162.0,"[['Always the best in bratislava', 'Very good ...",/Restaurant_Review-g274924-d1112354-Reviews-Ma...,d1112354


In [9]:
sample_submission.head(5)

,Restaurant_id,Rating
0,id_0,2.0
1,id_1,2.5
2,id_2,4.0
3,id_3,1.0
4,id_4,4.0


In [10]:
sample_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
Restaurant_id    10000 non-null object
Rating           10000 non-null float64
dtypes: float64(1), object(1)
memory usage: 156.4+ KB


In [11]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 11 columns):
Restaurant_id        50000 non-null object
City                 50000 non-null object
Cuisine Style        38410 non-null object
Ranking              50000 non-null float64
Price Range          32639 non-null object
Number of Reviews    46800 non-null float64
Reviews              49998 non-null object
URL_TA               50000 non-null object
ID_TA                50000 non-null object
sample               50000 non-null int64
Rating               50000 non-null float64
dtypes: float64(3), int64(1), object(7)
memory usage: 4.2+ MB


Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

In [13]:
data.sample(5)

,Restaurant_id,City,Cuisine Style,Ranking,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,sample,Rating
20270,id_524,Luxembourg,"['Mexican', 'Vegetarian Friendly']",525.0,$$ - $$$,27.0,"[['Surprisingly Good Food - Bad Music', ""Chain...",/Restaurant_Review-g190356-d12364690-Reviews-C...,d12364690,1,3.5
1273,id_1273,Athens,['Greek'],1702.0,$$$$,27.0,"[['Lovely food with very friendly service', 'N...",/Restaurant_Review-g189400-d3862721-Reviews-El...,d3862721,0,0.0
24138,id_1885,Vienna,['International'],1887.0,$,NaN,"[['Very nice!'], ['09/26/2016']]",/Restaurant_Review-g190454-d10805400-Reviews-B...,d10805400,1,5.0
20605,id_1576,Prague,"['Italian', 'Pizza', 'European', 'Vegetarian F...",1579.0,$$ - $$$,86.0,"[['Good food reasonable price', 'Surprisingly ...",/Restaurant_Review-g274707-d2392634-Reviews-Pi...,d2392634,1,3.5
687,id_687,Warsaw,"['Italian', 'Mediterranean', 'European', 'Pizz...",2006.0,$$ - $$$,36.0,"[['Nice place with fare prices', 'Rude staff a...",/Restaurant_Review-g274856-d9866008-Reviews-Pr...,d9866008,0,0.0


In [14]:
data.Reviews[1]

"[['Very good reviews!', 'Fine dining in Hakaniemi'], ['12/05/2017', '10/29/2017']]"

## 1. Обработка NAN 
У наличия пропусков могут быть разные причины, но пропуски нужно либо заполнить, либо исключить из набора полностью. Но с пропусками нужно быть внимательным, **даже отсутствие информации может быть важным признаком!**   
По этому перед обработкой NAN лучше вынести информацию о наличии пропуска как отдельный признак 

In [15]:
# Для примера возьмем столбец Number of Reviews
data['Number_of_Reviews_isNAN'] = pd.isna(data['Number of Reviews']).astype('uint8')

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 12 columns):
Restaurant_id              50000 non-null object
City                       50000 non-null object
Cuisine Style              38410 non-null object
Ranking                    50000 non-null float64
Price Range                32639 non-null object
Number of Reviews          46800 non-null float64
Reviews                    49998 non-null object
URL_TA                     50000 non-null object
ID_TA                      50000 non-null object
sample                     50000 non-null int64
Rating                     50000 non-null float64
Number_of_Reviews_isNAN    50000 non-null uint8
dtypes: float64(3), int64(1), object(7), uint8(1)
memory usage: 4.2+ MB


In [17]:
# Далее заполняем пропуски 0
data['Number of Reviews'].fillna(0, inplace=True)

### 2. Обработка признаков
Для начала посмотрим какие признаки у нас могут быть категориальными.

In [18]:
data.nunique(dropna=False)

Restaurant_id              13094
City                          31
Cuisine Style              10732
Ranking                    12975
Price Range                    4
Number of Reviews           1574
Reviews                    41858
URL_TA                     49963
ID_TA                      49963
sample                         2
Rating                        10
Number_of_Reviews_isNAN        2
dtype: int64

In [20]:
# Добавим столбец признак Восточная (=1)/Западная Европа (=0)
data["Eur"] = 0
East = {"Prague", "Budapest", "Warsaw","Krakow", "Bratislava", "Ljubljana"}
data.loc[data['City'].isin(East), 'Eur']=1


In [21]:
# для One-Hot Encoding в pandas есть готовая функция - get_dummies. 
data = pd.get_dummies(data, columns=[ 'City',], dummy_na=True)

In [22]:
data.head(5)

,Restaurant_id,Cuisine Style,Ranking,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,sample,Rating,...,City_Oporto,City_Oslo,City_Paris,City_Prague,City_Rome,City_Stockholm,City_Vienna,City_Warsaw,City_Zurich,City_nan
0,id_0,"['Bar', 'Pub']",12963.0,$$ - $$$,4.0,"[[], []]",/Restaurant_Review-g187147-d10746918-Reviews-L...,d10746918,0,0.0,...,0,0,1,0,0,0,0,0,0,0
1,id_1,"['European', 'Scandinavian', 'Gluten Free Opti...",106.0,$$ - $$$,97.0,"[['Very good reviews!', 'Fine dining in Hakani...",/Restaurant_Review-g189934-d6674944-Reviews-Ra...,d6674944,0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,id_2,['Vegetarian Friendly'],810.0,$$ - $$$,28.0,"[['Better than the Links', 'Ivy Black'], ['12/...",/Restaurant_Review-g186525-d13129638-Reviews-B...,d13129638,0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,id_3,"['Italian', 'Mediterranean', 'European', 'Vege...",1669.0,$$$$,202.0,"[['Most exquisite', 'Delicious and authentic']...",/Restaurant_Review-g186338-d680417-Reviews-Qui...,d680417,0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,id_4,"['Italian', 'Mediterranean', 'European', 'Seaf...",37.0,$$$$,162.0,"[['Always the best in bratislava', 'Very good ...",/Restaurant_Review-g274924-d1112354-Reviews-Ma...,d1112354,0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 44 columns):
Restaurant_id              50000 non-null object
Cuisine Style              38410 non-null object
Ranking                    50000 non-null float64
Price Range                32639 non-null object
Number of Reviews          50000 non-null float64
Reviews                    49998 non-null object
URL_TA                     50000 non-null object
ID_TA                      50000 non-null object
sample                     50000 non-null int64
Rating                     50000 non-null float64
Number_of_Reviews_isNAN    50000 non-null uint8
Eur                        50000 non-null int64
City_Amsterdam             50000 non-null uint8
City_Athens                50000 non-null uint8
City_Barcelona             50000 non-null uint8
City_Berlin                50000 non-null uint8
City_Bratislava            50000 non-null uint8
City_Brussels              50000 non-null uint8
City_Budapest  

In [24]:
data.sample(5)

,Restaurant_id,Cuisine Style,Ranking,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,sample,Rating,...,City_Oporto,City_Oslo,City_Paris,City_Prague,City_Rome,City_Stockholm,City_Vienna,City_Warsaw,City_Zurich,City_nan
13310,id_1578,"['Asian', 'Korean', 'Seafood', 'Sushi']",1580.0,$$ - $$$,22.0,"[['Bad sushi, bad service', 'Little Gem'], ['0...",/Restaurant_Review-g187331-d1320186-Reviews-Ki...,d1320186,1,4.0,...,0,0,0,0,0,0,0,0,0,0
22672,id_1789,['Healthy'],1803.0,NaN,6.0,"[[], []]",/Restaurant_Review-g187309-d3600817-Reviews-Mi...,d3600817,1,4.5,...,0,0,0,0,0,0,0,0,0,0
3907,id_3907,NaN,3865.0,NaN,11.0,"[[], []]",/Restaurant_Review-g187849-d6576966-Reviews-Gu...,d6576966,0,0.0,...,0,0,0,0,0,0,0,0,0,0
20298,id_1291,"['French', 'Bar', 'Cafe', 'European', 'Vegetar...",1292.0,$$ - $$$,151.0,"[[""Lovely brasserie close to Gare de L'est"", '...",/Restaurant_Review-g187147-d4291184-Reviews-Re...,d4291184,1,4.5,...,0,0,1,0,0,0,0,0,0,0
8003,id_8003,"['Steakhouse', 'Latin', 'Argentinean', 'South ...",663.0,$$ - $$$,424.0,[['Argentinian steak house with great atmosph....,/Restaurant_Review-g186338-d788787-Reviews-San...,d788787,0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
#
pr_dict = {'$': 1, '$$ - $$$': 2, '$$$$': 3}

#### Возьмем следующий признак "Price Range".

In [26]:
data['Price Range'].value_counts()

$$ - $$$    23041
$            7816
$$$$         1782
Name: Price Range, dtype: int64

По описанию 'Price Range' это - Цены в ресторане.  
Их можно поставить по возрастанию (значит это не категориальный признак). А это значит, что их можно заменить последовательными числами, например 1,2,3  
*Попробуйте сделать обработку этого признака уже самостоятельно!*

In [27]:
# Моя обработка 'Price Range'
pr_dict = {'$': 1, '$$ - $$$': 2, '$$$$': 3}

data = data.replace({"Price Range": pr_dict})

In [28]:
# А пропуски в 'Price Range' заполним наиболее часто встречающимися значениями. Предварительно создаем отдельную колонку и записываем где были пропуски.
data['Price Range_isNAN'] = pd.isna(data['Price Range']).astype('uint8')
data['Price Range'].fillna(2, inplace=True)

In [29]:
data['Price Range'].value_counts()

2.0    40402
1.0     7816
3.0     1782
Name: Price Range, dtype: int64

In [30]:
data.sample(5)

,Restaurant_id,Cuisine Style,Ranking,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,sample,Rating,...,City_Oslo,City_Paris,City_Prague,City_Rome,City_Stockholm,City_Vienna,City_Warsaw,City_Zurich,City_nan,Price Range_isNAN
29051,id_673,"['Italian', 'Bar', 'Fast Food', 'Mediterranean...",674.0,1.0,18.0,"[['Taste of Italy!!!', 'Really good!'], ['01/0...",/Restaurant_Review-g187331-d11938398-Reviews-F...,d11938398,1,5.0,...,0,0,0,0,0,0,0,0,0,0
44688,id_184,"['German', 'European', 'Central European']",185.0,1.0,180.0,"[['Hearty and wholesome with great portions.',...",/Restaurant_Review-g187331-d2304190-Reviews-Er...,d2304190,1,4.0,...,0,0,0,0,0,0,0,0,0,0
22224,id_9635,['French'],9637.0,2.0,15.0,"[[""Great eating place near Gare de l'ESt"", 'Ni...",/Restaurant_Review-g187147-d2501654-Reviews-Le...,d2501654,1,4.0,...,0,1,0,0,0,0,0,0,0,1
13026,id_6123,NaN,6127.0,2.0,5.0,"[[], []]",/Restaurant_Review-g187849-d13108922-Reviews-G...,d13108922,1,2.0,...,0,0,0,0,0,0,0,0,0,1
26393,id_5885,"['Japanese', 'Sushi', 'Asian']",5894.0,2.0,30.0,"[['Great Sushi !', 'OMG absolutely delicious']...",/Restaurant_Review-g186338-d3845005-Reviews-Oh...,d3845005,1,4.5,...,0,0,0,0,0,0,0,0,0,0


In [31]:
# тут ваш код на обработку других признаков
# Cuisine Style.....

In [32]:
from sklearn.preprocessing import LabelEncoder


# создаем отдельную колонку и запишем туда все пропуски, которые встречались в Cuisine Style
data['Cuisine Style_isNAN'] = pd.isna(data['Cuisine Style']).astype('float64') 

# Заполним пропуски значением 'Other' 
data['Cuisine Style'] = data['Cuisine Style'].fillna("['Other']")


In [33]:
# Формируем новые колонки исходя из кухонь в  Cuisine Style

from yaml import safe_load 
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer(sparse_output=True)

tmp = data["Cuisine Style"].dropna().apply(safe_load).dropna()

X = pd.DataFrame.sparse.from_spmatrix(
        mlb.fit_transform(tmp), 
        columns=mlb.classes_, 
        index=tmp.index)

data_new = data.join(X, how="left")

In [34]:
data_new.sample(5)

,Restaurant_id,Cuisine Style,Ranking,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,sample,Rating,...,Ukrainian,Uzbek,Vegan Options,Vegetarian Friendly,Venezuelan,Vietnamese,Welsh,Wine Bar,Xinjiang,Yunnan
3977,id_3977,"['Bar', 'Mediterranean', 'European', 'Spanish'...",966.0,2.0,729.0,"[['Tapas dinner', 'Great place to go for tapas...",/Restaurant_Review-g187514-d4004840-Reviews-La...,d4004840,0,0.0,...,0,0,0,1,0,0,0,0,0,0
45206,id_3818,"['Peruvian', 'South American']",3820.0,2.0,6.0,"[['Happy every time'], ['08/19/2017']]",/Restaurant_Review-g187514-d3812933-Reviews-Sa...,d3812933,1,4.5,...,0,0,0,0,0,0,0,0,0,0
27069,id_72,"['French', 'European', 'Asian', 'Contemporary'...",73.0,3.0,1155.0,"[['Fantastic', 'Beautifully Presented, Tastefu...",/Restaurant_Review-g187147-d720163-Reviews-Ze_...,d720163,1,4.5,...,0,0,0,1,0,0,0,0,0,0
13573,id_5460,['Italian'],5463.0,2.0,2.0,"[[], []]",/Restaurant_Review-g187849-d2185882-Reviews-Fu...,d2185882,1,4.0,...,0,0,0,0,0,0,0,0,0,0
33352,id_165,"['Austrian', 'European', 'Central European', '...",166.0,2.0,77.0,"[['Dinner', 'Great place'], ['10/21/2017', '05...",/Restaurant_Review-g190454-d4260452-Reviews-Wi...,d4260452,1,4.5,...,0,0,0,1,0,0,0,0,0,0


In [35]:
data_new.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 172 columns):
Restaurant_id              object
Cuisine Style              object
Ranking                    float64
Price Range                float64
Number of Reviews          float64
Reviews                    object
URL_TA                     object
ID_TA                      object
sample                     int64
Rating                     float64
Number_of_Reviews_isNAN    uint8
Eur                        int64
City_Amsterdam             uint8
City_Athens                uint8
City_Barcelona             uint8
City_Berlin                uint8
City_Bratislava            uint8
City_Brussels              uint8
City_Budapest              uint8
City_Copenhagen            uint8
City_Dublin                uint8
City_Edinburgh             uint8
City_Geneva                uint8
City_Hamburg               uint8
City_Helsinki              uint8
City_Krakow                uint8
City_Lisbon        

In [37]:
 data_new.iloc[:,46:]

,Afghani,African,Albanian,American,Arabic,Argentinean,Armenian,Asian,Australian,Austrian,...,Ukrainian,Uzbek,Vegan Options,Vegetarian Friendly,Venezuelan,Vietnamese,Welsh,Wine Bar,Xinjiang,Yunnan
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
49996,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
49997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [38]:
# Создаем отдельную колонку-признак - кол-во кухонь в ресторане
data_new["Count of Cuisines"] = data_new.iloc[:,46:].sum(axis=1)

In [39]:
#удаляем колонку Cuisine Style
data_new = data_new.drop(['Cuisine Style'], axis = 1)

# Reviews

In [ ]:
# В Reviews нет пропусков, но 6471 строк со значением [[], []]. По сути это пустые строки сохраним их 
data_new['empty_Reviews'] = (data_new['Reviews']=='[[], []]').astype('float64')

# анализ тестовой базы выявил два пропуска, несмотря на то, что pandas.profiling на тренировочной базе пропусков не выявил, заполним их '[[], []]' и закинем в empty_Reviews
data_new['Reviews'] = data_new['Reviews'].fillna('[[], []]')
data_new['empty_Reviews'] = (data_new['Reviews']=='[[], []]').astype('float64')


In [ ]:
data_new['date_of_Review'] = data_new['Reviews'].str.findall('\d+/\d+/\d+')
data_new['len_date'] = data_new['date_of_Review'].apply(lambda x: len(x))

In [ ]:
# есть значение 3 надо разобраться что там
print("кол-во значений Reviews с тремя датами :=" , len(data_new[data_new['len_date']==3]))
temp_list = data_new[data_new['len_date']==3].Reviews.to_list()
display(data_new[data_new['len_date']==3].Reviews.to_list())
print("даты после обработки регулярными выражениями:")
display([re.findall('\d+/\d+/\d+', x) for x in temp_list])

In [ ]:
# видим что люди указывали даты в отзывах и эти даты попали в обработку
# из-за этого возникнут ошибки так как даты не верные и их формат отличается и формата выгрузки
# при этом таких строк всего четыре (4), можно было бы их не исправлять а выбросить потому что 17 
# год явно приведет к выбросу с которым надо будет разбираться. Выбрасывать жалко, тогда исправим,
# тем более, что это достачно просто

data_new['len_date'].date_of_Review = data_new[data_new['len_date']==3].date_of_Review.apply(lambda x: x.pop(0))

In [ ]:
# также есть значение 1 надо разобраться что там
print("кол-во значений Reviews с одной датой :=" , len(data_new[data_new['len_date']==1]))
display(data_new[data_new['len_date']==1].Reviews[:4])

In [ ]:
# оказалось, что есть отзывы с одним (1) отзывом и их достаточно много 5680 из (40000-6471) это 17%
# сохраним это на всякий случай, чтобы не потерять
data_new['one_Review'] = (data_new['len_date']==1).astype('float64')

# заполним перерыв между отзывами (по отзывам где len = 2) и насколько давно был сделан последний самый свежий отзыв
# создадим для этого функции:
def time_to_now(row):
    if row['date_of_Review'] == []:
        return None
    return datetime.datetime.now() - pd.to_datetime(row['date_of_Review']).max()

def time_between_Reviews(row):
    if row['date_of_Review'] == []:
        return None
    return pd.to_datetime(row['date_of_Review']).max() - pd.to_datetime(row['date_of_Review']).min()

data_new['day_to_now'] = data_new.apply(time_to_now, axis = 1).dt.days
data_new['day_between_Reviews'] = data_new[data_new['len_date']==2].apply(time_between_Reviews, axis = 1).dt.days

Резюме - Reviews: Пропусков в тренировочном датасете нет, в тестовом - 2. Но есть 6471 незаполненных строк с отзывами в тренировочном датасете это 16% от датасета. В 5680 (14%) строках есть только один отзыв, хотя в подавляющем большинстве отзывов два. Созданы новые критерии:

empty_Reviews - незаполненные отзывы date_of_Review - даты из отзывов len_date - кол-во дат в отзыве day_to_now - насколько давно был сделан последний самый свежий отзыв в днях day_between_Reviews - перерыв между отзывами в днях Резюме - day_to_now из Reviews: Удаление по порогу не напрашивается так как компания TripAdvisor работает с 2000 года. Максимум 5896/365 ~ 16,5 лет от 2020 года укладывается в дату начала старта сайта. В выбросы попало 2365 (почти 6%) значений, с учетом резюме по неполным данным в критерии Reviews, я пока принимаю решение не избавлятся от выросов, построить модель, обратить внимание на важность критерия, и при необходимости вернуться к нему для заполнения парсингом или удаления выбросов

In [ ]:
# кол-во выбросов 495 (1.2%) - это статистически не значимо, но мы пока сохраняем информацию о выбросе, а потом проверим его важность в модели
data_new['out_day_between_Reviews'] = (data_new['day_between_Reviews']==0).astype('float64')

# и удаляем выбросы
data_new.loc[data_new['day_between_Reviews']==0, 'day_between_Reviews'] = None

In [ ]:
data_new.sample(5)

### Посмотрим распределение признака

In [ ]:
plt.rcParams['figure.figsize'] = (10,7)
df_train['Ranking'].hist(bins=100)

У нас много ресторанов, которые не дотягивают и до 2500 места в своем городе, а что там по городам?

In [ ]:
df_train['City'].value_counts(ascending=True).plot(kind='barh')

In [ ]:
df_train['Ranking'][df_train['City'] =='London'].hist(bins=100)

In [ ]:
# посмотрим на топ 10 городов
for x in (df_train['City'].value_counts())[0:10].index:
    df_train['Ranking'][df_train['City'] == x].hist(bins=100)
plt.show()

Получается, что Ranking имеет нормальное распределение, просто в больших городах больше ресторанов, из-за мы этого имеем смещение.



### Посмотрим распределение целевой переменной

In [ ]:
df_train['Rating'].value_counts(ascending=True).plot(kind='barh')

### Посмотрим распределение целевой переменной относительно признака

In [ ]:
df_train['Ranking'][df_train['Rating'] == 5].hist(bins=100)

In [ ]:
df_train['Ranking'][df_train['Rating'] < 4].hist(bins=100)

### И один из моих любимых - [корреляция признаков](https://ru.wikipedia.org/wiki/Корреляция)
На этом графике уже сейчас вы сможете заметить, как признаки связаны между собой и с целевой переменной.

In [ ]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(data.drop(['sample'], axis=1).corr(),)

Вообще благодаря визуализации в этом датасете можно узнать много интересных фактов, например:
* где больше Пицерий в Мадриде или Лондоне?
* в каком городе кухня ресторанов более разнообразна?

придумайте свои вопрос и найдите на него ответ в данных)

In [ ]:
    # убираем признаки которые еще не успели обработать, 
    # модель на признаках с dtypes "object" обучаться не будет, просто выберем их и удалим
    object_columns = [s for s in data_new.columns if data_new[s].dtypes == 'object']
    data_new.drop(object_columns, axis = 1, inplace=True)
    

In [ ]:
data_new.drop(['Restaurant_id', 'City', 'Cuisine Style', 'Price Range', 'Reviews', 'Number_of_Reviews_isNAN', 'URL_TA', 'ID_TA', 'date_of_Review', 'len_date', 'Сountry', 'Сity_population', 'mean_Ranking_on_City', 'count_Restorant_in_City', 'max_Ranking_on_City', ], axis=1, inplace=True, errors='ignore')

#### Запускаем и проверяем что получилось

In [ ]:
data_new.info()

In [ ]:
data_new.sample(2)

#  **Стандартизируем и заполняем нулями пропуски по всем переменным¶**

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

In [ ]:
# функция для стандартизации
def StandardScaler_column(d_col):
    scaler = StandardScaler()
    scaler.fit(data_new[[d_col]])
    return scaler.transform(data_new[[d_col]])
# стандартизируем все столбцы кроме целевой и Sample
for i  in list(data_new.columns):
    if i not in ['Rating','sample']:
        data_new[i] = StandardScaler_column(i)
        if len(data_new[data_new[i].isna()]) < len(data_new):
            data_new[i] = data_new[i].fillna(0)


In [ ]:
# Теперь выделим тестовую часть
train_data = data_new.query('sample == 1').drop(['sample'], axis=1)
test_data = data_new.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Rating'], axis=1)

In [ ]:
X.info()

**Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.**

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

# Submission
Если все устраевает - готовим Submission на кагл

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['Rating'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission

In [ ]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

# What's next?
Или что делать, чтоб улучшить результат:
* Обработать оставшиеся признаки в понятный для машины формат
* Посмотреть, что еще можно извлечь из признаков
* Сгенерировать новые признаки
* Подгрузить дополнительные данные, например: по населению или благосостоянию городов
* Подобрать состав признаков

В общем, процесс творческий и весьма увлекательный! Удачи в соревновании!
